# Toplogy of Deep Neural Networks

This notebook will show you how easy it is to use gdeep to reproduce the experiments of the paper [Topology of Deep Neural Networks](https://arxiv.org/pdf/2004.06093.pdf), by Naizat et. al. In this work, the authors studied the evolution of the topology of a dataset as embedded in the successive layers of a Neural Network, trained for classification on this dataset.

Their main findings can be summarized as follows:

- Neural networks tend to simplify the topology of the dataset accross layers.

- This decrease in topological complexity is more efficient when the activation functions are non-homeomorphic, as it is the case for ReLu or leakyReLu.

Here is an illustration from the paper:

![img](./images/topology_accross_layers.png)

The main steps of this tutorial will be as follows:

1. Create the Entangled Tori dataset.
2. Build several fully connected networks, with different activation functions.
3. Train these networks to classify the Entangled Tori datasets.
4. Visualise in tensorboard the persistence diagrams of the dataset embedded in each layers of each network.
5. Study the decrease in topological complexity of the dataset accross layers



## Import relevant librairies

In [22]:
%reload_ext autoreload
%autoreload 2

# deep learning
import torch
from torch.optim import Adam, SGD
import numpy as np
from torch import nn
from torch import autograd  

#gdeep
from gdeep.data.datasets import DatasetBuilder, DataLoaderBuilder
from gdeep.models import FFNet
from gdeep.visualisation import persistence_diagrams_of_activations
from gdeep.data.preprocessors import ToTensorImage
from gdeep.trainer import Trainer
from gdeep.search import Benchmark
from gdeep.search import GiottoSummaryWriter



# plot
import plotly.express as px
import pandas as pd


writer = GiottoSummaryWriter()

# ML
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_blobs
from sklearn.metrics import pairwise_distances

# TDA
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram

#Tensorboard

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


# Initialize the tensorboard writer

In order to analyse the reuslts of your models, you need to start tensorboard.
On the terminal, move inside the `/example` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.


# Generate the Entangled Tori dataset and prepare the dataloaders

![img](./images/entangled_tori.png)


In [23]:
from torch.utils.data import  RandomSampler
db = DatasetBuilder(name="EntangledTori")
ds_tr, ds_val, ds_ts = db.build( n_pts = 50)
dl_tr, dl_val, dl_ts = DataLoaderBuilder((ds_tr, ds_val, ds_ts)).build(    
     [{"batch_size":100, "sampler":RandomSampler(ds_tr)}, 
     {"batch_size":100, "sampler":RandomSampler(ds_tr)}, 
     {"batch_size":100, "sampler":RandomSampler(ds_tr)}]
     )

# Define models with different activations functions

In [24]:
import torch.nn.functional as F

# Choose the achitecture of the fully connected network
architecture = [3,5,5,5,2]
# Choose the loss function for training
loss_function = nn.CrossEntropyLoss()
# Choose the set of activation functions to equip the neural network with
activation_string = ["relu", "leakyrelu", "tanh", "sigmoid"]
activation_functions = [F.relu, F.leaky_relu, torch.tanh, torch.sigmoid]

In [25]:
# Define the models and trainers
models = []
writers = []
trainers = []
for i in range(len(activation_functions)):
    model_temp = FFNet(arch = architecture, activation = activation_functions[i])
    writer_temp = GiottoSummaryWriter(log_dir='runs/' + model_temp.__class__.__name__ + activation_string[i])
    trainer_temp = Trainer(model_temp, [dl_tr, dl_ts], loss_function, writer_temp)
    models.append(model_temp)
    writers.append(writer_temp)
    trainers.append(trainer_temp)

# Let's train our models!

You can monitor the training in the tensorboard page

In [26]:
for pipe in trainers:
    pipe.train(
    Adam,
    10,
    False,
    {"lr": 0.01},
    {"batch_size": 200})

Epoch 1
-------------------------------
Epoch training loss: 0.619556 	Epoch training accuracy: 57.38%                                                             / 160 ]                     
Time taken for this epoch: 1.00s
Learning rate value: 0.01000000


/home/berkouknicolas/giotto-deep/gdeep/trainer/trainer.py:455: UserWarning:

Cannot store data in the PR curve



Validation results: 
 accuracy: 63.54%,                 Avg loss: 0.569001 

Epoch 2
-------------------------------
Epoch training loss: 0.557137 	Epoch training accuracy: 63.68%                                                             8.5  	[ 59 / 160 ]                     
Time taken for this epoch: 1.00s
Learning rate value: 0.01000000
Validation results: 
 accuracy: 66.28%,                 Avg loss: 0.544553 

Epoch 3
-------------------------------
Epoch training loss: 0.533706 	Epoch training accuracy: 66.52%                                                            
Time taken for this epoch: 1.00s
Learning rate value: 0.01000000
Validation results: 
 accuracy: 66.25%,                 Avg loss: 0.535445 

Epoch 4
-------------------------------
Epoch training loss: 0.518293 	Epoch training accuracy: 67.78%                                                            
Time taken for this epoch: 1.00s
Learning rate value: 0.01000000
Validation results: 
 accuracy: 67.42%,      

# For each model, let's plot the topology of the dataset embedded in each layer of the network

We start by the Betti curves. For a subset of size batch_size of the dataset, we compute the successive Betti numbers of the Vietoris-Rips complex of radius filtration_value of the subset embedded in each layer of the network. The result is plotted in tensorboard.

In [28]:
from gdeep.visualisation import Visualiser
#Choose the size of the subset of the dataset
batch_size = 1000
#Extract the subset of the dataset
one_batch_dataset, _, _ = DataLoaderBuilder((ds_tr, ds_val, ds_ts)).build(
    [{"batch_size":batch_size, "sampler":RandomSampler(ds_tr)}, 
    {"batch_size":batch_size, "sampler":RandomSampler(ds_tr)}, 
    {"batch_size":batch_size, "sampler":RandomSampler(ds_tr)}]) 

#For each model, plot the Betti curve
for pipe in trainers:
    vs = Visualiser(pipe)
    vs.plot_betti_numbers_layers(homology_dimensions = [0,1], 
        batch = next(iter(one_batch_dataset)), 
        filtration_value = 0.5)





If you have a bit more time, you can even compute the Persistence Diagrams of the subset of the dataset embedded in each layers, and plot them in tensorboard! The computation might take a few minutes.

In [ ]:
for pipe in trainers:
    vs = Visualiser(pipe)
    vs.plot_persistence_diagrams(next(iter(one_batch_dataset)), k= 0)
